<a href="https://colab.research.google.com/github/daje0601/ml_pipline_study/blob/main/3_wandb_minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.7 MB 4.1 MB/s 
     |████████████████████████████████| 140 kB 46.3 MB/s 
     |████████████████████████████████| 97 kB 5.9 MB/s 
     |████████████████████████████████| 180 kB 64.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
import wandb 
wandb.init(project="wandb-tutorial",
           config={
                  "layer_1_activation":"relu", 
                  "layer_1":128,
                   "learning_rate":0.01,
                   "dropout_rate": 0.2
            })

In [33]:
# 위 설정한 config를 config라는 변수에 담아주도록 하겠습니다. 
config = wandb.config

In [34]:
# wandb에 로그를 찍기 위해서 tensorflow와 wandb를 연결해주어야 합니다. 
# 이때 사용되는 라이브러리가 WandbCallback이고 이를 model.fit에 넣어주면 됩니다. 
from wandb.keras import WandbCallback

In [35]:
import tensorflow as tf 

minst = tf.keras.datasets.mnist 
(train_x, train_y), (test_x, test_y) = minst.load_data()

In [36]:
train_x.shape

(60000, 28, 28)

In [37]:
train_x, test_x = train_x / 255.0, test_x / 255.0

In [40]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer_1, activation=config.layer_1_activation), # 기존에는 128이라는 직접적인 숫자를 넣어주었지만, 이제는 config.layer_1이라고 수정합니다. 
    tf.keras.layers.Dropout(config.dropout_rate), 
    tf.keras.layers.Dense(10, activation="softmax")
])

In [41]:
# 기존에는 optimizer="adam", 기재하였으나 이는 learning_rate를 직접 지정할 수 없습니다. 
# optimzier=tf.keras.optimizers.Adam(learning_rate=config.learning_rate)라고 수정해줍니다. 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate), 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [42]:
model.fit(train_x, train_y, epochs=5, callbacks=[WandbCallback()])

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3216 - accuracy: 0.9050
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2461 - accuracy: 0.9299
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2270 - accuracy: 0.9383
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2138 - accuracy: 0.9436
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2029 - accuracy: 0.9443


In [43]:
model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 1ms/step - loss: 0.1933 - accuracy: 0.9586


[0.1933409869670868, 0.9585999846458435]